# Experimentos y análisis


In [1]:
import pandas as pd
import subprocess
import os

CPU_EXEC = "../build/src/MyProjectCPU"
CL_EXEC = "../build/src/cl/MyProjectCL"
CUDA_EXEC = "../build/src/cuda/MyProjectCUDA"

## Experimento 1
$N$ variable con `block_size` y `grid_size` fijos

- Párametros:
    - $N$: De 10 a 100000

In [2]:

current_path = os.getcwd()
block = 1024
grid = 8

for i in range(1, 11):
    size = 2**i
    print("Running for size", i)
    # do 10 runs for each size
    for _ in range(10):
        subprocess.run([CPU_EXEC, str(size), current_path + "/cpu.csv"], stdout=subprocess.DEVNULL)
        subprocess.run([CL_EXEC, str(size), str(block), str(grid*block), current_path + "/cl.csv"], stdout=subprocess.DEVNULL)
        subprocess.run([CUDA_EXEC, str(size), str(block), str(grid*block), current_path + "/cuda.csv"], stdout=subprocess.DEVNULL)

Running for size 1


TypeError: expected str, bytes or os.PathLike object, not int